In [ ]:
from flask import Flask,render_template,request
import pandas as pd
from jinja2 import Template
import viola
import os
import base64
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt1
from sklearn.cluster import KMeans
import seaborn as sns
import sklearn.metrics as sma
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.impute import SimpleImputer
from IPython.display import Image
import dataframe_image as dfi
import pickle
app=Flask(__name__)
#weather=pickle.load(open('weather.pkl','rb'))
@app.route("/")
@app.route("/html")
@app.route("/kmeans.html")
def html():
    return render_template("dates-wise1.html")


@app.route('/result',methods=['POST'])

@app.route('/plot.png')


def result():
        if request.form['submit']=='Predict':
        
            Date1=request.form['Date']
            df = pd.read_csv('C:/Users/HP/Desktop/Weather_Prediction/Weather-Prediction/Final Weather Dataset.csv')
            #Take 4 pollutant column
            X= df.iloc[:,[2,3,4,5]].values
            #Replacing missing values with median
            s_imputer=SimpleImputer(missing_values=0,strategy="mean")
            s_imputer=s_imputer.fit(X)
            X=s_imputer.transform(X)
            y=pd.DataFrame(X)

            #Calculate value of k
            wcss = []
            for k in range(1,11):
                kmeans = KMeans(n_clusters=k,init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
                kmeans.fit(X)
                wcss.append(kmeans.inertia_)
            
            kmeans = KMeans(n_clusters =4, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
            kmeans.fit_predict(y)

            centroids=kmeans.cluster_centers_
            centroid_df=pd.DataFrame(centroids,columns=list(y))
            y_kmeans=pd.DataFrame(kmeans.labels_,columns=list(['labels']))
            y_kmeans['labels']=y_kmeans['labels'].astype('category')
            df_labeled=df.join(y_kmeans)
            df_analysis=(df_labeled.groupby(['labels'],axis=0)).tail(2332)
            df_analysis['labels'] = np.where((df_analysis.labels =="0"),"dusty,fly ash smogy,hot,fog,mist",df_analysis.labels)
            df_analysis['labels'] = np.where((df_analysis.labels =="1"),"hot,dry,smogy",df_analysis.labels)
            df_analysis['labels'] = np.where((df_analysis.labels =="2"),"hot,smogy,humid",df_analysis.labels)
            df_analysis['labels'] = np.where((df_analysis.labels =="3"),"hot,smogy",df_analysis.labels)


            #df_analysis.Date= pd.to_datetime(df_analysis.Date)
            d=df_analysis[['Date','labels']].query('Date == @Date1')
            return render_template("dates-wise1.html",prediction=d)
        
        elif request.form['submit']=='Cluster':
            
            Date1=request.form['Date']
            df = pd.read_csv('C:/Users/HP/Desktop/Weather_Prediction/Weather-Prediction/Final Weather Dataset.csv')
            X= df.iloc[:,[2,3,4,5]].values
            
            #Replacing missing values with median
            s_imputer=SimpleImputer(missing_values=0,strategy="mean")
            s_imputer=s_imputer.fit(X)
            X=s_imputer.transform(X)
            
            y=pd.DataFrame(X)
            wcss = []
            for k in range(1,11):
                kmeans = KMeans(n_clusters=k,init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
                kmeans.fit(X)
                wcss.append(kmeans.inertia_)
            
            kmeans = KMeans(n_clusters =4, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
            kmeans.fit_predict(y)

            centroids=kmeans.cluster_centers_
            centroid_df=pd.DataFrame(centroids,columns=list(y))
            y_kmeans=pd.DataFrame(kmeans.labels_,columns=list(['labels']))
            y_kmeans['labels']=y_kmeans['labels'].astype('category')
            df_labeled=df.join(y_kmeans)
            df_analysis=(df_labeled.groupby(['labels'],axis=0)).tail(2332)
            
            df_analysis['labels'] = np.where((df_analysis.labels =="0"),"dusty,fly ash smogy,hot,fog,mist",df_analysis.labels)
            df_analysis['labels'] = np.where((df_analysis.labels =="1"),"hot,dry,smogy",df_analysis.labels)
            df_analysis['labels'] = np.where((df_analysis.labels =="2"),"hot,smogy,humid",df_analysis.labels)
            df_analysis['labels'] = np.where((df_analysis.labels =="3"),"hot,smogy",df_analysis.labels)

            df_analysis.Date= pd.to_datetime(df_analysis.Date)
            d=df_analysis[['Date','labels']].query('Date == @Date1')
            kmeans = KMeans(n_clusters =4, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
            y_kmeans= kmeans.fit_predict(y)
           
            plt.scatter(X[y_kmeans==0,0],X[y_kmeans==0,1],s=50,c="red",label="Label:RPM")
            plt.scatter(X[y_kmeans==1,0],X[y_kmeans==1,1],s=50,c="blue",label="Label:NOx")
            plt.scatter(X[y_kmeans==2,0],X[y_kmeans==2,1],s=50,c="green",label="Label:CO2")
            plt.scatter(X[y_kmeans==3,0],X[y_kmeans==3,1],s=50,c="magenta",label="Label:SO2")

            plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:, 1], color = 'yellow',label = 'Centroids',s=300)
           


            plt.title('Clusters for user entered date using Particular Matters present in Air')
            plt.legend()
            imagepath1=os.path.join('static','image'+'.png')
            plt.savefig(imagepath1)
            
            return render_template("image.html",image=imagepath1)
      


    
if __name__=='__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [22/May/2021 21:14:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/May/2021 21:14:36] "GET /css/bootstrap.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/May/2021 21:14:36] "GET /css/bootstrap-responsive.css HTTP/1.1" 404 -
ERROR:__main__:Exception on /result [POST]
Traceback (most recent call last):
  File "c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\flask\app.py", line 2051, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\flask\app.py", line 1501, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\flask\app.py", line 1499, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\flask\app.py", li